# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symmer.utils import unit_n_sphere_cartesian_coords
from symmer.symplectic import PauliwordOp, QuantumState, array_to_QuantumState

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

1.000 |0> -> 0.000+1.000j |1>
1.000 |1> -> 0.000-1.000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state: 1.000 |00>

After application of the Hadamard gate:

0.500+0.000j |00> +
0.500+0.000j |01> +
0.500+0.000j |10> +
0.500-0.000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

0.500+0.000j ZZ +
0.500+0.000j ZX +
0.500+0.000j XZ +
0.500-0.000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

0.175+0.016j |1011110001> +
0.115+0.315j |0110101001> +
0.340+0.048j |0001100001> +
0.124+0.042j |1011000011> +
0.135+0.310j |0110000110> +
0.191+0.251j |0100010001> +
0.057+0.299j |1111001001> +
0.218+0.005j |0001000001> +
0.317+0.218j |0101111000> +
0.331+0.332j |1011100001>

0.182+0.092j |1011111001> +
0.355+0.416j |0001010101> +
0.298+0.345j |1111001010> +
0.233+0.255j |1110110110> +
0.227+0.149j |1011100010> +
0.148+0.164j |0010000010> +
0.176+0.096j |1100010111> +
0.173+0.217j |1011110110> +
0.078+0.259j |0000101001> +
0.127+0.053j |0001011010>


In [7]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

0.055+0.183j |0000101001> +
0.154+0.003j |0001000001> +
0.251+0.294j |0001010101> +
0.090+0.038j |0001011010> +
0.241+0.034j |0001100001> +
0.104+0.116j |0010000010> +
0.135+0.178j |0100010001> +
0.224+0.154j |0101111000> +
0.095+0.219j |0110000110> +
0.081+0.223j |0110101001> +
0.088+0.030j |1011000011> +
0.234+0.235j |1011100001> +
0.160+0.106j |1011100010> +
0.123+0.011j |1011110001> +
0.122+0.154j |1011110110> +
0.129+0.065j |1011111001> +
0.125+0.068j |1100010111> +
0.165+0.180j |1110110110> +
0.040+0.211j |1111001001> +
0.211+0.244j |1111001010>

Norm: (0.9999999999999998+0j)


In [8]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

0.913+0.000j XXXXZXIZII +
1.287+0.000j YZZZIIXZXI +
0.786+0.000j XXIIXXYXYZ +
-25.848+0.000j YIXXYZYXZZ +
2.264+0.000j ZYZXIZYIXZ +
0.442+0.000j IXXXIIXIYY +
-0.039+0.000j YZIIXYXYXZ +
-1.272+0.000j IZIXYZXXIZ +
0.777+0.000j XZXZIIYZZY +
1.095+0.000j ZIYIIYXIXY

Expectation value <psi|op|psi> = (-0.030281407293039492+1.734723475976807e-18j)


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [9]:
statevector = np.zeros(2**10)
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = array_to_QuantumState(statevector)
print(psi)
print(psi.conjugate * psi)

0.408 |0000000000> +
0.577 |0011111010> +
0.707 |0111110100>
1.0
